<a href="https://colab.research.google.com/github/arun-mishra22/Data-Science-Interview-Assistant/blob/main/model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
print(torch.cuda.is_available())


True


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
!pip install transformers peft datasets accelerate bitsandbytes evaluate rouge_score



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6f18c200e1786bb193f6b9bf9c85cdc93294fc8642688304d073dd257a4e0f0b
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

from peft import PeftModel
import torch

base_path = "/content/drive/MyDrive/LLM_Base_Models/Llama3B_base"
adapter_path = "/content/drive/MyDrive/LLM_Base_Models/Llama3B_lora_adapter"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(base_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


base_model = AutoModelForCausalLM.from_pretrained(
    base_path,
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, adapter_path)

print("Fine-tuned model loaded!")


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:246: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading weights:   0%|          | 0/254 [00:00<?, ?it/s]

Fine-tuned model loaded!


In [11]:
from datasets import Dataset

test_dataset = Dataset.from_file("/content/test.arrow")

print(test_dataset)


Dataset({
    features: ['text'],
    num_rows: 479
})


In [12]:
print(test_dataset.column_names)
print(test_dataset[0])


['text']
{'text': '### Instruction:\nIs it possible to create a GPT model without a GPU?\n\n### Response:\nYes, it is possible to create a GPT model without a GPU. GPUs are used to improve the speed at which a model runs, however it is possible to use a lower-end processor or a much slower CPU to run a GPT model.'}


In [13]:
def split_prompt_and_answer(text):
    parts = text.split("### Response:")

    if len(parts) == 2:
        prompt = parts[0] + "### Response:"
        answer = parts[1].strip()
        return prompt, answer
    else:
        return text, ""


In [14]:
def generate_answer(prompt_list):

    inputs = tokenizer(
        prompt_list,
        return_tensors="pt",
        padding="longest",
        truncation=True
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150
    )

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)


In [15]:
prompts = []
references = []

for item in test_dataset:
    prompt, true_answer = split_prompt_and_answer(item["text"])
    prompts.append(prompt)
    references.append(true_answer)

predictions = []

batch_size = 8

for i in range(0, len(prompts), batch_size):

    batch = prompts[i : i + batch_size]

    preds = generate_answer(batch)

    predictions.extend(preds)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [26]:
import pandas as pd

df = pd.DataFrame({
    "prompt": prompts,
    "reference": references,
    "prediction": predictions
})

df.to_csv("model_predictions.csv", index=False)

print("Saved results to model_predictions.csv")


Saved results to model_predictions.csv


In [16]:
print(len(predictions))
print(len(references))


479
479


In [17]:
cleaned_predictions = []

for p in predictions:
    # If model returns prompt + answer together, keep only answer part
    if "### Response:" in p:
        cleaned_predictions.append(p.split("### Response:")[-1].strip())
    else:
        cleaned_predictions.append(p.strip())


In [18]:
correct = 0

for pred, ref in zip(cleaned_predictions, references):
    if pred.strip().lower() == ref.strip().lower():
        correct += 1

accuracy = correct / len(references)

print("Exact Match Accuracy:", accuracy)


Exact Match Accuracy: 0.006263048016701462


In [19]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smoothie = SmoothingFunction().method4

scores = []

for pred, ref in zip(cleaned_predictions, references):
    score = sentence_bleu(
        [ref.split()],
        pred.split(),
        smoothing_function=smoothie
    )
    scores.append(score)

print("Average BLEU Score:", sum(scores) / len(scores))



Average BLEU Score: 0.05102039022656898


In [20]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

rouge_scores = []

for pred, ref in zip(cleaned_predictions, references):
    score = scorer.score(ref, pred)
    rouge_scores.append(score['rougeL'].fmeasure)

print("Average ROUGE-L Score:", sum(rouge_scores) / len(rouge_scores))


Average ROUGE-L Score: 0.1751907713106612


In [22]:
!pip install bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.4 MB/s eta 0:00:00


In [24]:
!pip install --upgrade transformers tokenizers evaluate bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0.0


Direct manual testing

In [32]:
from transformers import pipeline, AutoTokenizer, GenerationConfig

tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# FIX: Explicitly set pad token
tokenizer.pad_token = tokenizer.eos_token

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

gen_config = GenerationConfig(
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

question = "What is bias variance tradeoff? How can you avoid it?"

prompt = f"""### Instruction:
{question}

### Response:
"""

output = pipe(
    prompt,
    generation_config=gen_config,
    return_full_text=False   # VERY IMPORTANT
)

print(output[0]["generated_text"])



This is a core concept in ML theory and practice. Understanding it helps build robust systems and choose appropriate ML approaches. Key dimensions: computational cost, generalization performance, interpretability vs. lack of explanatory power. Alternatives include Bayesian approach (intractable), empirical risk minimization (ERM), early stopping, data augmentation, architecture search, multi-task learning, ensemble methods, model averaging, and other approaches.


In [33]:
!pip install bert-score


In [34]:
from evaluate import load

bertscore = load("bertscore")

bert_results = bertscore.compute(
    predictions=predictions,
    references=references,
    lang="en"
)

print("Average BERTScore F1:", sum(bert_results["f1"]) / len(bert_results["f1"]))


Loading weights:   0%|          | 0/389 [00:00<?, ?it/s]

RobertaModel LOAD REPORT from: roberta-large
Key                             | Status     | 
--------------------------------+------------+-
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
pooler.dense.bias               | MISSING    | 
pooler.dense.weight             | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


AttributeError: RobertaTokenizer has no attribute build_inputs_with_special_tokens

In [39]:
!pip uninstall -y transformers tokenizers bert-score evaluate



Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: tokenizers 0.15.2
Uninstalling tokenizers-0.15.2:
  Successfully uninstalled tokenizers-0.15.2
Found existing installation: bert-score 0.3.13
Uninstalling bert-score-0.3.13:
  Successfully uninstalled bert-score-0.3.13
Found existing installation: evaluate 0.4.6
Uninstalling evaluate-0.4.6:
  Successfully uninstalled evaluate-0.4.6


In [ ]:
!pip install transformers==4.37.2
!pip install tokenizers==0.15.0
!pip install bert-score==0.3.13
!pip install evaluate==0.4.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 13.7 MB/s eta 0:00:00
  Using cached tokenizers-0.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 56.5 MB/s eta 0:00:00
Using cached tokenizers-0.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.2.2 requires transformers<6.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 46.3 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C


In [7]:
import pandas as pd

df = pd.read_csv("model_predictions.csv")

predictions = df["prediction"].tolist()
references = df["reference"].tolist()

print("Loaded samples:", len(predictions))


Loaded samples: 479


In [8]:
from evaluate import load

bertscore = load("bertscore")

bert_results = bertscore.compute(
    predictions=predictions,
    references=references,
    lang="en"
)

print("Average BERTScore F1:", sum(bert_results["f1"]) / len(bert_results["f1"]))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average BERTScore F1: 0.844624831557025


In [5]:
!pip install bert-score


  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
Using cached bert_score-0.3.13-py3-none-any.whl (61 kB)


In [2]:
!pip install evaluate



  Using cached evaluate-0.4.6-py3-none-any.whl.metadata (9.5 kB)
Using cached evaluate-0.4.6-py3-none-any.whl (84 kB)


In [3]:
from evaluate import load


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]